In [1]:
import os
import json
#import cPickle as pickle
import pickle
from tqdm import tqdm
import logging

import os.path as op

In [3]:
data_path = "/home/zmykevin/fb_intern/data/vinvl_data/recoco+/refcoco+"
refcoco_plus = json.load(open(os.path.join(data_path, "instances.json"), "r"))

In [4]:
# for k,v in refcoco_plus.items():
#     print(k)
#     print(v)
#     break
print(refcoco_plus.keys())
print(type(refcoco_plus['images']))
print(type(refcoco_plus['annotations']))
print(type(refcoco_plus['categories']))

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])
<class 'list'>
<class 'list'>
<class 'list'>


In [5]:
for x in refcoco_plus['images']:
    print(x)
    break
refcoco_plus_image = {}
for x in refcoco_plus['images']:
    image_id = x["id"]
    refcoco_plus_image[image_id] = True

{'license': 1, 'file_name': 'COCO_train2014_000000098304.jpg', 'coco_url': 'http://mscoco.org/images/98304', 'height': 424, 'width': 640, 'date_captured': '2013-11-21 23:06:41', 'flickr_url': 'http://farm6.staticflickr.com/5062/5896644212_a326e96ea9_z.jpg', 'id': 98304}


In [6]:
for y in refcoco_plus['annotations']:
    print(y)
    break
    
refcoco_plus_ann = {}
for y in refcoco_plus['annotations']:
    ann_id = y['id']
    refcoco_plus_ann[ann_id] = {"bbox": y['bbox'], "image_id": y["image_id"]}
#     print(refcoco_plus_ann)
#     break

{'segmentation': [[267.52, 229.75, 265.6, 226.68, 265.79, 223.6, 263.87, 220.15, 263.87, 216.88, 266.94, 217.07, 268.48, 221.3, 272.32, 219.95, 276.35, 220.15, 279.62, 218.03, 283.46, 218.42, 285.0, 220.92, 285.0, 223.22, 284.42, 224.95, 280.96, 225.14, 279.81, 226.48, 281.73, 228.41, 279.43, 229.37, 275.78, 229.17, 273.86, 229.56, 274.24, 232.05, 269.82, 231.67, 267.14, 231.48, 266.75, 228.6]], 'area': 197.29899999999986, 'iscrowd': 0, 'image_id': 98304, 'bbox': [263.87, 216.88, 21.13, 15.17], 'category_id': 18, 'id': 3007}


In [7]:
print(len(refcoco_plus['images']))

19992


In [36]:
# for z in refcoco_plus['categories']:
#     print(z)
#     break

In [8]:
ref_file = os.path.join(data_path, 'refs('+'unc'+').p')
#reference = pickle.load(open(ref_file, 'r'))
reference = pickle.load(open(ref_file, "rb"))

In [9]:
print(type(reference))

<class 'list'>


In [10]:
for z in reference:
    print(z)
    break

{'sent_ids': [0, 1, 2], 'file_name': 'COCO_train2014_000000581857_16.jpg', 'ann_id': 1719310, 'ref_id': 0, 'image_id': 581857, 'split': 'train', 'sentences': [{'tokens': ['navy', 'blue', 'shirt'], 'raw': 'navy blue shirt', 'sent_id': 0, 'sent': 'navy blue shirt'}, {'tokens': ['woman', 'back', 'in', 'blue'], 'raw': 'woman back in blue', 'sent_id': 1, 'sent': 'woman back in blue'}, {'tokens': ['blue', 'shirt'], 'raw': 'blue shirt', 'sent_id': 2, 'sent': 'blue shirt'}], 'category_id': 1}


In [11]:
#Annotation Include: file_name; referring_expression; Ground_truth_Box_Id: 
print(len(refcoco_plus['images']))
print(len(refcoco_plus['annotations']))
print(len(reference))

19992
196737
49856


In [12]:
split = []
for x in reference:
    c_split = x['split']
    if c_split not in split:
        split.append(c_split)
print(split)

['train', 'testB', 'testA', 'val']


In [13]:
#Load the COCO VinVL features
class TSVFile(object):
    def __init__(self, tsv_file, generate_lineidx=False):
        self.tsv_file = tsv_file
        self.lineidx = op.splitext(tsv_file)[0] + '.lineidx'
        self._fp = None
        self._lineidx = None
        # the process always keeps the process which opens the file. 
        # If the pid is not equal to the currrent pid, we will re-open the file.
        self.pid = None
        # generate lineidx if not exist
        if not op.isfile(self.lineidx) and generate_lineidx:
            generate_lineidx_file(self.tsv_file, self.lineidx)

    def __del__(self):
        if self._fp:
            self._fp.close()

    def __str__(self):
        return "TSVFile(tsv_file='{}')".format(self.tsv_file)

    def __repr__(self):
        return str(self)

    def num_rows(self):
        self._ensure_lineidx_loaded()
        return len(self._lineidx)

    def seek(self, idx):
        self._ensure_tsv_opened()
        self._ensure_lineidx_loaded()
        try:
            pos = self._lineidx[idx]
        except:
            logging.info('{}-{}'.format(self.tsv_file, idx))
            raise
        self._fp.seek(pos)
        return [s.strip() for s in self._fp.readline().split('\t')]

    def seek_first_column(self, idx):
        self._ensure_tsv_opened()
        self._ensure_lineidx_loaded()
        pos = self._lineidx[idx]
        self._fp.seek(pos)
        return read_to_character(self._fp, '\t')

    def __getitem__(self, index):
        return self.seek(index)

    def __len__(self):
        return self.num_rows()

    def _ensure_lineidx_loaded(self):
        if self._lineidx is None:
            logging.info('loading lineidx: {}'.format(self.lineidx))
            with open(self.lineidx, 'r') as fp:
                self._lineidx = [int(i.strip()) for i in fp.readlines()]

    def _ensure_tsv_opened(self):
        if self._fp is None:
            self._fp = open(self.tsv_file, 'r')
            self.pid = os.getpid()

        if self.pid != os.getpid():
            logging.info('re-open {} because the process id changed'.format(self.tsv_file))
            self._fp = open(self.tsv_file, 'r')
            self.pid = os.getpid()
            
            
def prepare_image_keys(tsv):
    return [tsv.seek(i)[0] for i in range(tsv.num_rows())]

In [16]:
coco_data_path = "/home/zmykevin/fb_intern/data/vinvl_data/coco"
coco_feature_tsv = os.path.join(coco_data_path, "features.tsv")
coco_prediction_tsv = os.path.join(coco_data_path, "predictions.tsv")

In [17]:
feat_tsv = TSVFile(coco_feature_tsv)
pred_tsv = TSVFile(coco_prediction_tsv)

In [18]:
relevant_pred = {}
for i in tqdm(range(pred_tsv.num_rows())):
    #print(pred_tsv.seek(i))
    current_pred = pred_tsv.seek(i)
    image_id = int(current_pred[0])
    #check if the image_id is in the image list
    if refcoco_plus_image.get(image_id, None) is not None:
        relevant_pred[image_id] = current_pred[1]
        #print(current_pred)
        #break
print(len(relevant_pred))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123287/123287 [00:02<00:00, 51332.36it/s]

19992


In [19]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [20]:
#Load the corresponding vinvl features:
split_category = ["train", "testB", "testA", "val"]
split = "val"

refcoco_plus_final = []
for x in tqdm(reference):
    new_data_point = {}
    #current_split = x['split']
    #Process the data
    #if current_split == split:
    current_split = x['split']
    image_file_name = "_".join(x['file_name'].split("_")[:-1])+".jpg"
    image_id = int(image_file_name.split('_')[2].split('.')[0])
    #Find the corresponding annotation
    ann_id = x['ann_id']
    current_annotation = refcoco_plus_ann[ann_id]
    #(xtl,ytl,weidth,height)
    #print(current_annotation["bbox"])
    gt_bbox = [current_annotation["bbox"][0],current_annotation["bbox"][1],
               current_annotation["bbox"][0]+current_annotation["bbox"][2],
               current_annotation["bbox"][1]+current_annotation["bbox"][3]
              ]
        
    #print(gt_bbox)
    #verify the image id matches
    assert current_annotation['image_id'] == image_id
    #Get the corresponding vinvl_prediction
    vinvl_pred = json.loads(relevant_pred[image_id])
    max_iou = 0
    max_index = 0
    valid_index = []
    for i, obj in enumerate(vinvl_pred["objects"]):
        candidate_bbox = obj["rect"]
        #compute iou
        current_iou = bb_intersection_over_union(candidate_bbox, gt_bbox)
        if current_iou > max_iou:
            max_iou = current_iou
            max_index = i
        if current_iou > 0.5:
            valid_index.append(i)
    #Only take the iou larger than 0.5
    if max_iou > 0.5 and max_index < 100:
        new_data_point["file_name"] = image_file_name
        new_data_point["image_id"] = image_id
        new_data_point["ann_id"] = ann_id
        new_data_point["bbox"] = current_annotation["bbox"]
        new_data_point["gt_proposal"] = max_index
        new_data_point["valid_proposals"] = valid_index
        new_data_point["sentences"] = x["sentences"]
        new_data_point["split"] = current_split
        refcoco_plus_final.append(new_data_point)
    #print(new_data_point)
    #break
        

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49856/49856 [00:13<00:00, 3819.30it/s]


In [21]:
print(len(refcoco_plus_final))
more_gt = 0
max_gt = 0
for x in refcoco_plus_final:
    valid_proposals = x['valid_proposals']
    if len(valid_proposals) > 1:
        more_gt += 1
    if len(valid_proposals) > max_gt:
        max_gt = len(valid_proposals)

print(more_gt)
print(max_gt)
        

48352
24945
7


In [22]:
pad_id = 100
pad_length = 7
#Pad the Index 
for x in refcoco_plus_final:
    x["valid_proposals"] += (pad_length-len(x["valid_proposals"]))*[pad_id]

In [23]:
#Save this augmented reference
output_path = os.path.join(data_path,"refcoco_plus_final.json")
with open(output_path, "w") as f:
    json.dump(refcoco_plus_final, f)